In [1]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 
import pickle
import warnings


from utils import *

from sensor_params import *

from models.training import *

from models.multi_step.unet1D_regressor import *
from models.multi_step.unet1D_regressor import get_model as get_unet1D

# from models.multi_step.unet1D_DIL_regressor import *
# from models.multi_step.unet1D_DIL_regressor import get_model as get_DIL_unet1D

from models.multi_step.LSTMVanilla import *
from models.multi_step.LSTMVanilla import get_model as get_LSTM

from models.multi_step.StackedLSTM import *
from models.multi_step.StackedLSTM import get_model as get_StackedLSTM

# from models.multi_step.unet1D_LSTM_regressor import *
# from models.multi_step.unet1D_LSTM_regressor import get_model as get_unet1DLSTM

# from models.multi_step.unet1D_nonCompres_regressor import *
# from models.multi_step.unet1D_nonCompres_regressor import get_model as get_unet1D_nonCompres

from evaluation.multi_step.evaluation import model_evaluation

# Dataset path 
DATABASE_PATH = r"C:\Users\aralmeida\OneDrive - Universidad de Las Palmas de Gran Canaria\Doctorado\Bases de datos\WARIFA\Mikael T1DM"

filename = "MIKAEL_data.json"

parent_directory = r"C:\Users\aralmeida\Downloads"
experiments_folder = r"\T1DM_pred_experiments" 

In [2]:
# Load data from sgv_data. If executed, comment the line and load the data from the pickle file
# data , basal_data_dict, blood_pressure_dict, bolus_data_dict, exercise_data_dict, carbohydrates_data_dict, pump_events_data_dict, sgv_data_dict, sleep_data_dict, smgb_data_dict, steps_data_dict, weight_dict  = extract_Mikael_data(DATABASE_PATH, filename, ONLY_CGM = True)
with open(DATABASE_PATH+'\CGM.pk1', 'rb') as f:
    sgv_data_dict = pickle.load(f)
os.chdir(parent_directory)

In [3]:
# Dataset generation parameters
N = 144
step = 1
PH = 5
pred_steps = round(PH/sensor_Mikael["SAMPLE_PERIOD"])
single_multi_step = 'multi_step' # 'single' or 'multi'
data_partition = 'june-21' # could be 'month-wise
normalization = 'min-max'
under_over_sampling = 'under'
name = '1DUNET_himar-rep'

In [4]:
update_results_dictionary(parent_directory, experiments_folder, single_multi_step, N, step, PH, data_partition, normalization, under_over_sampling, name) 

Non-existing dictionary. A new one was created.

Dictionary entry created.



In [5]:
N = 72
step = 1
PH = 5
pred_steps = round(PH/sensor_Mikael["SAMPLE_PERIOD"])
single_multi_step = 'multi_step' # 'single' or 'multi'
data_partition = 'june-21' # could be 'month-wise
normalization = 'min-max'
under_over_sampling = 'under'
name = '1DUNET_himar-rep'

In [6]:
update_results_dictionary(parent_directory, experiments_folder, single_multi_step, N, step, PH, data_partition, normalization, under_over_sampling, name) 

Dictionary loaded.

Dictionary entry created.



In [7]:
N = 72
step = 1
PH = 60
pred_steps = round(PH/sensor_Mikael["SAMPLE_PERIOD"])
single_multi_step = 'multi_step' # 'single' or 'multi'
data_partition = 'june-21' # could be 'month-wise
normalization = 'min-max'
under_over_sampling = 'under'
name = '1DUNET_himar-rep'

In [8]:
update_results_dictionary(parent_directory, experiments_folder, single_multi_step, N, step, PH, data_partition, normalization, under_over_sampling, name) 

Dictionary loaded.

Dictionary entry created.



In [9]:
with open(parent_directory+experiments_folder+r'\results_dictionary.json', 'rb') as handle:
    results_dictionary = json.load(handle)
    print("Dictionary loaded.\n")

results_dictionary

Dictionary loaded.



{'multi_step_N144_step1_PH5_june-21_min-max_under_1DUNET_himar-rep': {'RMSE': 0,
  'MAE': 0,
  'MAPE': 0,
  'ISO': 0,
  'Parker': 0,
  'time_lag': 0},
 'multi_step_N72_step1_PH5_june-21_min-max_under_1DUNET_himar-rep': {'RMSE': 0,
  'MAE': 0,
  'MAPE': 0,
  'ISO': 0,
  'Parker': 0,
  'time_lag': 0},
 'multi_step_N72_step1_PH60_june-21_min-max_under_1DUNET_himar-rep': {'RMSE': 0,
  'MAE': 0,
  'MAPE': 0,
  'ISO': 0,
  'Parker': 0,
  'time_lag': 0}}

In [ ]:
X, Y, X_times, Y_times = get_CGM_X_Y_multistep(sgv_data_dict, sensor_Mikael, N, step, PH, experiments_folder, plot=False, verbose = 0)

In [ ]:
# Min-max normalization
if normalization == 'min-max':
    X_norm = (X - np.min(X))/(np.max(X) - np.min(X))
    Y_norm = (Y - np.min(X))/(np.max(X) - np.min(X))
elif normalization == None: 
    X_norm = X
    Y_norm = Y

In [ ]:
if data_partition == 'june-21':
    
    # Simple partition (imitating Himar work for comparison): X_train until 30/05/2021 and X_test from 31/05/2021
    # Instances that include two days are removed
    X_train = X_norm[np.where(X_times[:,N-1] <= pd.to_datetime('2021-05-31 00:00:00'))[0]]
    Y_train = Y_norm[np.where((Y_times[:,0] < pd.to_datetime('2021-05-30 23:59:59')))[0]]
    X_test = X_norm[np.where((X_times[:,N-1] > pd.to_datetime('2021-06-01 00:00:00')))[0]]
    Y_test = Y_norm[np.where((Y_times[:,0] > pd.to_datetime('2021-06-01 00:00:00')))[0]] # Left non-normalized to compute the metrics

    print("X_train shape: ",str(X_train.shape))
    print("Y_train shape: ",str(Y_train.shape))
    print("X_test shape: ",str(X_test.shape))
    print("Y_test shape: ",str(Y_test.shape))

elif data_partition == 'month-wise-4-folds':
    pass

In [ ]:
if under_over_sampling == 'under':
    X_train, Y_train  = undersample_normal_range_outputs(X, X_train, Y_train, multi_step = True, normalization = normalization, undersampling_factor = 2)
else : 
    pass

In [ ]:
# Parameters for training
# Best parameters for the model N 48
lr = 0.0001
batch_size = 2
epochs = 10

# Model hyperparameters
kernel_size = 3
stride = 1

# lr = 0.001
# batch_size = 2
# epochs = 10
# kernel_size = 3

In [ ]:
lr = 0.0001
batch_size = 256
epochs = 1

In [ ]:
# Get model instance depending on the name
################### CNNs/ UNETs ###################
if name == '1DUNET_himar-rep':
    model =  get_unet1D(sensor = sensor_Mikael,
                    N=N,
                    input_features = NUMBER_OF_INPUT_SIGNALS,
                    tau=stride,
                    kernel_size=kernel_size,
                    PH=PH)

elif name == '1DUNET-non-compres_himar-rep':
    # model =  get_unet1D_nonCompres(N=N,
    #                 input_features = NUMBER_OF_INPUT_SIGNALS,
    #                 tau=tau,
    #                 kernel_size=kernel_size,
    #                 predicted_points=1)
    pass

elif name == '1DUNET-DIL_himar-rep':
    # model =  get_DIL_unet1D(N=N,
    #                 input_features = NUMBER_OF_INPUT_SIGNALS,
    #                 tau=tau,
    #                 kernel_size=kernel_size,
    #                 dilation_rate=1,  
    #                 predicted_points=1)
    pass

################### LSTMs ###################
elif name == 'LSTM_himar-rep':
    model =  get_LSTM(sensor = sensor_Mikael,
                    N=int(N),
                    input_features = NUMBER_OF_INPUT_SIGNALS,
                    PH=PH)

elif name == 'StackedLSTM_himar-rep':
    model =  get_StackedLSTM(sensor = sensor_Mikael,
                    N=N,
                    input_features = NUMBER_OF_INPUT_SIGNALS,
                    PH=PH)


################### Hybryds ###################
elif name == '1DUNET-LSTM_himar-rep':
    # model =  get_unet1DLSTM(N=N,
    #                 input_features = NUMBER_OF_INPUT_SIGNALS,
    #                 tau=tau,
    #                 kernel_size=kernel_size,
    #                 predicted_points=1)
    pass

else: 
    raise ValueError("Model name not valid")

In [ ]:
train_model(sensor_Mikael,
            model,
            X = X_train,
            Y = Y_train,
            N = N,
            predicted_points = pred_steps, 
            epochs = epochs,
            batch_size = batch_size,
            lr = lr,
            verbose = 1, 
            fold = name)

In [ ]:
model_evaluation(N, PH, name, normalization, X_test, Y_test, pred_steps, X) 